<a href="https://colab.research.google.com/github/zineb99hmd/Ant-Colony-Optimization/blob/master/CoronaVirus_Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Télecharger dataset

In [32]:
data=pd.read_excel('/dataset.xlsx')
df=data.copy()
df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1. Exploratory Data Analysis
Objectif :

*   Comprendre du mieux possible nos données(un petit pas en avant vaut mieux qu'un grand pas en arriere)
*  Développer une premiere stratégie de modélisation
# Checklist de base
> **Analyse de Forme :**


*   variable target : SARS-Cov-2 exam result
*   lignes et colonnes : 5644, 111


*   types de variables : qualitatives : 70, quantitatives : 41
*   Analyse des valeurs manquantes :
beaucoup de NaN (moitié des variables > 90% de NaN)
2 groupes de données 76% -> Test viral, 89% -> taux sanguins


> **Analyse de Fond :**


*   Visualisation de la target :
10% de positifs (558 / 5000)
*   Signification des variables :

 -variables continues standardisées, skewed (asymétriques), test sanguin.

 -age quantile : difficile d'interpreter ce graphique, clairement ces données ont été traitées, on pourrait penser 0-5, mais cela pourrait aussi etre une transformation mathématique. On peut pas savoir car la personne qui a mit ce dataset ne le précise nul part. Mais ca n'est pas tres important.

 -variable qualitative : binaire (0, 1), viral, Rhinovirus qui semble tres élevée



* **Relation Variables / Target :**

-target / blood : les taux de Monocytes, Platelets, Leukocytes semblent liés au covid-19 -> hypothese a tester

-target/age : les individus de faible age sont tres peu contaminés ? -> attention on ne connait pas l'age, et on ne sait pas de quand date le dataset (s'il s'agit des enfants on sait que les enfants sont touchés autant que les adultes). En revanche cette variable pourra etre intéressante pour la comparer avec les résultats de tests sanguins

-target / viral : les doubles maladies sont tres rares. Rhinovirus/Enterovirus positif - covid-19 négatif ? -> hypothese a tester ? mais il est possible que la région est subie une épidémie de ce virus. De plus on peut tres bien avoir 2 virus en meme temps. Tout ca n'a aucun lien avec le covid-19

# **Analyse plus détaillée**
* **Relation Variables / Variables :**

*   blood_data / blood_data : certaines variables sont tres corrélées : +0.9 (a suveiller plus tard)

*  blood_data / age : tres faible corrélation entre age et taux sanguins
*  viral / viral : influenza rapid test donne de mauvais résultats, il fauda peut-etre la laisser tomber
*  relation maladie / blood data : Les taux sanguins entre malades et covid-19 sont différents
* relation hospitalisation / est malade :
*  relation hospitalisation / blood : intéressant dans le cas ou on voudrait prédire dans quelle service un patient devrait aller
* NaN analyse : viral : 1350(92/8), blood : 600(87/13), both : 90
* **hypotheses nulle (H0):**
* Les individus atteints du covid-19 ont des taux de Leukocytes, Monocytes, Platelets significativement différents

* H0 = Les taux moyens sont ÉGAUX chez les individus positifs et négatifs
* Les individus atteints d'une quelconque maladie ont des taux significativement différents



# Analyse de la forme des données

In [33]:
df.shape

(5644, 111)